In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q /content/drive/MyDrive/milk_and_eggs_final.zip

In [4]:
%cd /content/images

/content/images


In [5]:
import os
len(os.listdir('/content/images'))

381

In [ ]:
!pip install --upgrade Pillow

     |████████████████████████████████| 3.1 MB 14.9 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import PIL
PIL.__version__

'9.1.1'

In [ ]:
from PIL import Image, ImageOps
import glob
import os

path = '/content/zzz/' #the path where to save resized images

if not os.path.exists(path):
    os.makedirs(path)
new_height = 800

for filename in os.listdir('/content/images/'): #path of raw images
    img = Image.open(filename)
    new_width = int(new_height / img.height * img.width)
    img.thumbnail((new_width , new_height), Image.ANTIALIAS)
    # # save resized images to new folder with existing filename
    img.save('{}{}{}'.format(path,'/',os.path.split(filename)[1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  


In [ ]:
len(os.listdir('/content/milk_and_eggs_resized_final'))

381

In [ ]:
%cd ../

/content


In [ ]:
!zip -r milk_and_eggs_resized_final.zip /content/milk_and_eggs_resized_final

  adding: content/milk_and_eggs_resized_final/ (stored 0%)
  adding: content/milk_and_eggs_resized_final/714c17fb-19094beb-IMG-20210929-WA0040.jpg (deflated 1%)
  adding: content/milk_and_eggs_resized_final/ea46cf47-31bc2926-IMG_20211003_145204.jpg (deflated 1%)
  adding: content/milk_and_eggs_resized_final/97e8d4f4-7b367a03-IMG_20211003_145252.jpg (deflated 1%)
  adding: content/milk_and_eggs_resized_final/8ae8a514-8da61634-IMG_20211002_122020.jpg (deflated 0%)
  adding: content/milk_and_eggs_resized_final/f47bc385-fbaf373f-IMG_20211003_145213.jpg (deflated 0%)
  adding: content/milk_and_eggs_resized_final/2f3e2a7e-photo_2022-03-25_21-36-21.jpg (deflated 0%)
  adding: content/milk_and_eggs_resized_final/8bbd0826-IMG_0158.JPG (deflated 0%)
  adding: content/milk_and_eggs_resized_final/4697dc32-IMG_0166.JPG (deflated 0%)
  adding: content/milk_and_eggs_resized_final/a6df392c-3bb926c3-IMG_8995.JPG (deflated 0%)
  adding: content/milk_and_eggs_resized_final/53106927-06651b16-IMG_20210928_

In [ ]:
from google.colab import files
files.download("/content/milk_and_eggs_resized_final.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Modeling

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 13702, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 13702 (delta 8), reused 9 (delta 4), pack-reused 13683
Receiving objects: 100% (13702/13702), 12.18 MiB | 7.54 MiB/s, done.
Resolving deltas: 100% (9571/9571), done.


In [3]:
!unzip -q /content/drive/MyDrive/milk_and_eggs_final_resized.zip

In [4]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 25.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import torchvision
import os 
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from IPython.display import Image
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split

In [6]:
images = [os.path.join('images', x) for x in os.listdir('images')]
annotations = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

In [7]:
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.1)
# val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state=42)

In [8]:
!mkdir train
!mkdir val
# !mkdir test 

!mkdir train/images/ 
!mkdir val/images/ 
# !mkdir test/images/

!mkdir train/labels/ 
!mkdir val/labels/ 
# !mkdir test/labels/

In [11]:
# Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'train/images/')
move_files_to_folder(val_images, 'val/images/')
# move_files_to_folder(test_images, 'test/images/')
move_files_to_folder(train_annotations, 'train/labels/')
move_files_to_folder(val_annotations, 'val/labels/')
# move_files_to_folder(test_annotations, 'test/labels/')

In [ ]:
classes = open("classes.txt", "r")

In [ ]:
all_classes = classes.read().split('\n')

In [ ]:
len(all_classes)

138

In [ ]:
all_classes

['Молоко Parmalat 1,8% ультрапастеризованное',
 'Молоко Parmalat 3,5% ультрапастеризованное',
 'Молоко Parmalat Comfort 0,05% ультрапастеризованное',
 'Молоко Parmalat Dietalat 0,5% ультрапастеризованное',
 'Молоко «Ярослава» пастеризованное 3,2%, 930 мл',
 'Молоко Авида 2,5% ультрапастеризованное',
 'Молоко Авида 3,2% пастеризованное',
 'Молоко Авида 3,2% ультрапастеризованное',
 'Молоко Агуша 2.5%',
 'Молоко Амка ультрапастеризованное 3,2%',
 'Молоко Асеньевская ферма 2,5% пастеризованное',
 'Молоко Асеньевская ферма 3,4%-6.0%',
 'Молоко Асеньевская ферма отборное пастеризованное',
 'Молоко Асеньевская ферма топленое',
 'Молоко БМК (Брянского молочного комбината) 2,5%',
 'Молоко БМК (Брянского молочного комбината) 3,2%',
 'Молоко Белый Город ультрапастеризованное 2,5%',
 'Молоко Белый Город ультрапастеризованное 3,2%',
 'Молоко Бидончик 3,2% ультрапастеризованное',
 'Молоко Брест-Литовск 3,6% пастеризованное',
 'Молоко Бурёнкино Отборное',
 'Молоко Валио 1,5% ультрапастеризованное',


# Training

In [12]:
sorted(os.listdir('/content/train/images'))[:5]

['00626cd0-IMG_0175_JPG.rf.fb3ae2b4368d78795e09860272a77349.jpg',
 '046c5b2b-10cd9431-IMG_1012_JPG.rf.ebd90feed7d933f16a0ddb7975217c05.jpg',
 '047f4225-0676534a-IMG-20210929-WA0080_jpg.rf.5d5e0e82351757a99e79ea29d0bb954a.jpg',
 '067a5de3-IMG_20220512_202504_jpg.rf.967df3b56d9f2f5fc712b280be9dcba4.jpg',
 '072fa948-IMG_0219_JPG.rf.d3a7196330193d1a2414c523eba54572.jpg']

In [13]:
sorted(os.listdir('/content/train/labels'))[:5]

['00626cd0-IMG_0175.txt',
 '046c5b2b-10cd9431-IMG_1012.txt',
 '047f4225-0676534a-IMG-20210929-WA0080.txt',
 '067a5de3-IMG_20220512_202504.txt',
 '072fa948-IMG_0219.txt']

In [14]:
renames = sorted(os.listdir('/content/train/images'))
for index, filename in enumerate(sorted(os.listdir('/content/train/labels'))):
  os.rename(f'/content/train/labels/{filename}', f'/content/train/labels/{renames[index]}')

In [15]:
for filename in os.listdir('/content/train/labels'):
  os.rename(f'/content/train/labels/{filename}', f'/content/train/labels/{filename.replace(".jpg", ".txt")}')

In [16]:
renames = sorted(os.listdir('/content/val/images'))
for index, filename in enumerate(sorted(os.listdir('/content/val/labels'))):
  os.rename(f'/content/val/labels/{filename}', f'/content/val/labels/{renames[index].replace(".jpg", ".txt")}')

In [17]:
!pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 33.0 MB/s 
     |████████████████████████████████| 181 kB 66.4 MB/s 
     |████████████████████████████████| 145 kB 64.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [19]:
%cd yolov5

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5


In [20]:
!python train.py --img 640 --batch 16 --epochs 300 --data /content/yolov5/data/coco128.yaml --weights /content/best_450_epochs.pt --cache

wandb: Currently logged in as: deepone. Use `wandb login --relogin` to force relogin
train: weights=/content/best_450_epochs.pt, cfg=, data=/content/yolov5/data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-221-gd07f9ce Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, 

KeyboardInterrupt: ignored

In [ ]:
cd ../

/content


In [ ]:
!zip -r milk_and_eggs_resized_final_train.zip /content/train
!zip -r milk_and_eggs_resized_final_val.zip /content/val

  adding: content/train/ (stored 0%)
  adding: content/train/images/ (stored 0%)
  adding: content/train/images/6f983abb-9077d2b2-IMG_9009_JPG.rf.cd0dea91de20f5c298e4a5047f428c06.jpg (deflated 0%)
  adding: content/train/images/78766465-photo_2022-03-25_21-36-04_jpg.rf.9890d40241191db9b21c53da50d10665.jpg (deflated 1%)
  adding: content/train/images/799c6860-ec710c27-IMG_9032_JPG.rf.34ac3714297566441e50e0812753d521.jpg (deflated 0%)
  adding: content/train/images/6517a310-28394c6f-IMG_20210930_132415_jpg.rf.5caa9f052ac408fd986e0103e285d53a.jpg (deflated 0%)
  adding: content/train/images/2e092f32-02629a79-IMG_8997_JPG.rf.c999e8b571567dd982530e1e870a0aea.jpg (deflated 1%)
  adding: content/train/images/8a883ff1-fcd46d6f-IMG_20211002_121830_jpg.rf.29afdb253e08eb2a835fc42887af03e8.jpg (deflated 0%)
  adding: content/train/images/d355536e-6d197e4c-IMG_20210928_130909_jpg.rf.6cbffdf521a475070f27ab00f7ad145d.jpg (deflated 0%)
  adding: content/train/images/f25c8ad0-IMG_0186_JPG.rf.86a363501d

In [ ]:
from google.colab import files
files.download("/content/milk_and_eggs_resized_final_train.zip")
files.download("/content/milk_and_eggs_resized_final_val.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>